In [1]:
from datetime import timedelta as delta
from parcels import FieldSet, AdvectionRK4
import numpy as np
from parcels import ParticleSetSOA, ScipyParticle, JITParticle

npart = 10000

f = 1.e-4
u_0 = 0.3
u_g = 0.04
gamma = 1/(86400. * 2.89)
gamma_g = 1/(86400. * 28.9)

In [2]:
def fieldset_stationary(xdim=100, ydim=100, maxtime=delta(hours=6)):
    """Generate a FieldSet encapsulating the flow field of a stationary eddy.

    Reference: N. Fabbroni, 2009, "Numerical simulations of passive
    tracers dispersion in the sea"
    """
    time = np.arange(0., maxtime.total_seconds()+1e-5, 60., dtype=np.float64)
    dimensions = {'lon': np.linspace(0, 25000, xdim, dtype=np.float32),
                  'lat': np.linspace(0, 25000, ydim, dtype=np.float32),
                  'time': time}
    data = {'U': np.ones((xdim, ydim, 1), dtype=np.float32) * u_0 * np.cos(f * time),
            'V': np.ones((xdim, ydim, 1), dtype=np.float32) * -u_0 * np.sin(f * time)}
    return FieldSet.from_data(data, dimensions, mesh='flat', transpose=True)

In [3]:
ptype = ScipyParticle
fieldset = fieldset_stationary()
lon = np.linspace(12000, 21000, npart)
lat = np.linspace(12500, 12500, npart)
pset = ParticleSetSOA(fieldset, pclass=ptype, lon=lon, lat=lat)
endtime = delta(hours=6).total_seconds()


In [4]:
%%time
pset.execute(AdvectionRK4, dt=delta(minutes=25), endtime=endtime)


CPU times: user 1min 4s, sys: 380 ms, total: 1min 4s
Wall time: 1min 4s


In [5]:
ptype = JITParticle
fieldset = fieldset_stationary()
lon = np.linspace(12000, 21000, npart)
lat = np.linspace(12500, 12500, npart)
pset = ParticleSetSOA(fieldset, pclass=ptype, lon=lon, lat=lat)
endtime = delta(hours=6).total_seconds()


In [6]:
%%time
pset.execute(AdvectionRK4, dt=delta(seconds=1), endtime=endtime)


INFO: Compiled ArrayJITParticleAdvectionRK4 ==> /tmp/parcels-1000/lib4b6e96b2215a4775e084478cefaa750b_0.so


CPU times: user 1min 4s, sys: 9.14 ms, total: 1min 4s
Wall time: 1min 5s
